# Step 4

In [1]:
from neo4j import GraphDatabase

# Setting up database connection
uri = "bolt://localhost:7687"
username = "neo4j"
password = "Whatever your password is"
driver = GraphDatabase.driver(uri, auth=(username, password))


# Step 5

In [2]:
pii_query = """
CALL gds.graph.project(
    'clientClusters',
    { Client: {
        label: 'Client'
    }},
    { SHARED_PII: {
        type: 'SHARED_PII',
        orientation: 'UNDIRECTED',
        properties: {
            count: {
                property: 'count'
            }
        }
    }}
)
YIELD graphName, nodeCount, relationshipCount
"""
with driver.session() as session:
    session.run(pii_query)



# Step 6

In [3]:
#In-memory projection of graph
with driver.session() as session:
    session.run("""
        CALL gds.graph.project(
        'clientClusters',
        'Client',
        'SHARED_PII'
        )
    """)



# Step 7

In [4]:
# WCC
with driver.session() as session:
    session.run("""
        CALL gds.wcc.stream(
            'clientClusters',
            {
                nodeLabels: ['Client'],
                relationshipTypes: ['SHARED_PII'],
                consecutiveIds: true
            }
        )
        YIELD nodeId, componentId
        RETURN gds.util.asNode(nodeId).id AS clientId, componentId
        ORDER BY componentId, clientId
    """)

# Step 8

In [5]:
with driver.session() as session:
    session.run("""
        CALL gds.wcc.stream(
            'clientClusters',
            {
                nodeLabels: ['Client'],
                relationshipTypes: ['SHARED_PII'],
                consecutiveIds: true
            }
        )
        YIELD nodeId, componentId
        WITH gds.util.asNode(nodeId) AS clientId, componentId AS clusterId
        WITH clusterId, collect(clientId.id) AS clients
        WITH clusterId, clients, size(clients) AS clusterSize 
        WHERE clusterSize >= 2
        UNWIND clients AS client
        MATCH (c:Client) WHERE c.id = client
        SET c.secondPartyFraudRing = clusterId
    """)

# Step 9

In [6]:
#Cypher projection with script from slides
with driver.session() as session:
    session.run("""
        // first, find clients
            MATCH (c:Client) WHERE c.secondPartyFraudRing is NOT NULL
            WITH collect(c) as clients
        // second, find the PII nodes
            MATCH (n) WHERE n:Email OR n:Phone OR n:SSN
        // combine the two sets of nodes
            WITH clients, collect(n) AS piis
            WITH clients + piis AS nodes
        // use only the clients that belong to a cluster of size >= 2
        // as per STEP 8
            MATCH (c:Client) -[:HAS_EMAIL | :HAS_PHONE | :HAS_SSN]->(p)
            WHERE c.secondPartyFraudRing is NOT NULL
        // now make the bipartite graph,
        // with a new relationship named HAS_PII
            WITH nodes, collect({source: c, target: p}) as relationships
        // use a Cypher projection
        // not the usual native projection
            CALL gds.graph.project.cypher(
                'similarity',
                "UNWIND $nodes as n
                RETURN id(n) AS id,labels(n) AS labels",
                    "UNWIND $relationships as r
                RETURN id(r['source']) AS source, id(r['target']) AS target,
                    'HAS_PII' as type",
                { parameters:
                { nodes: nodes,
                    relationships: relationships }
                }
            )
        YIELD graphName, nodeCount, relationshipCount
        RETURN graphName, nodeCount, relationshipCount
    """)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The semantics of using colon in the separation of alternative relationship types will change in a future version. (Please use ':HAS_EMAIL|HAS_PHONE|HAS_SSN' instead)} {position: line: 12, column: 56, offset: 512} for query: '\n        // first, find clients\n            MATCH (c:Client) WHERE c.secondPartyFraudRing is NOT NULL\n            WITH collect(c) as clients\n        // second, find the PII nodes\n            MATCH (n) WHERE n:Email OR n:Phone OR n:SSN\n        // combine the two sets of nodes\n            WITH clients, collect(n) AS piis\n            WITH clients + piis AS nodes\n        // use only the clients that belong to a cluster of size >= 2\n        // as per STEP 8\n            MATCH (c:Client) -[:HAS_EMAIL | :HAS_PHONE | :HAS_SSN]

# Step 10

In [7]:
with driver.session() as session:
    session.run("""
    CALL gds.nodeSimilarity.mutate('similarity', {
      mutateProperty: 'jaccardScore',
      mutateRelationshipType: 'SIMILAR_TO',
      topK: 15
    })
    YIELD nodesCompared, relationshipsWritten
    """)

# Step 11

In [8]:
with driver.session() as session:
    # Repeat step 10 in new session
    # Had to make a separate relationship to copy it to the original db,
    # as it couldn't be called the same thing as in step 10
    session.run("""
    CALL gds.nodeSimilarity.mutate('similarity', {
      mutateProperty: 'jaccardScore',
      mutateRelationshipType: 'COPY_SIMILAR_TO', 
      topK: 15
    })
    YIELD nodesCompared, relationshipsWritten
    """)

    # NOW writing SIMILAR_TO back to og database
    session.run("""
    CALL gds.graph.writeRelationship('similarity', 'SIMILAR_TO', 'jaccardScore')
    YIELD relationshipsWritten
    """)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated procedure. ('gds.graph.writeRelationship' has been replaced by 'gds.graph.relationships.write')} {position: line: 2, column: 1, offset: 5} for query: "\n    CALL gds.graph.writeRelationship('similarity', 'SIMILAR_TO', 'jaccardScore')\n    YIELD relationshipsWritten\n    "


# Step 12

In [9]:
with driver.session() as session:
    session.run("""
    CALL gds.degree.write(
      'similarity', 
      {
        nodeLabels: ['Client'],
        relationshipTypes: ['SIMILAR_TO'],
        relationshipWeightProperty: 'jaccardScore',
        writeProperty: 'secondPartyFraudScore'
      })
    YIELD centralityDistribution, nodePropertiesWritten
    """)

# Step 13

In [10]:
with driver.session() as session:
    session.run("""
    MATCH (c:Client)
    WHERE c.secondPartyFraudScore IS NOT NULL
    WITH percentileCont(c.secondPartyFraudScore, 0.95) AS threshold
    MATCH (c:Client)
    WHERE c.secondPartyFraudScore > threshold
    SET c:SecondPartyFraudster
    """)

# Step 14

In [11]:
#For dataframe to print out results
import pandas as pd
with driver.session() as session:
    result = session.run("""
    MATCH (c:Client)
    WHERE c:SecondPartyFraudster
    RETURN c.name AS c_name, c.id AS c_id
    """)
    
    df = pd.DataFrame(result, columns=['c_name', 'c_id'])
    
    print(df)

               c_name              c_id
0   Brooklyn Harrison  4029043591201321
1         Aiden Hurst  4583937317122539
2         Sarah Klein  4912587051525728
3       Kennedy Keith  4446118457512030
4    Samantha Mueller  4717521340444448
5       Benjamin Moss  4189330002136246
6    Charlotte Foster  4024985944102082
7          Ryan Patel  4614177132519923
8        Landon Welch  4359490519123048
9     Allison Alvarez  4371660075922934
10     Madeline Ramos  4818802026065667
11     Gabriel Oliver  4385217169131833
12   Scarlett Solomon  4830783673717400
13       Julia Ortega  4632977841783696
14      Jose Roberson  4268433407129628
15   Aaliyah Thornton  4934732209995365
16        Emily Stout  4748268948389491
